In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [2]:
df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
df_test = pd.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")
df_concepts = pd.read_csv("/root/kag_comp/UM_MCTS/Data/concepts.csv")

In [123]:
def checking_set(col, start, end) :
    """
    포함하는 계층형 구조를 지닌 행들을 식별할 때 사용
    """
    lst = list(df_train.columns)[start:end]
    if (df_train.loc[df_train[col] == 1,].iloc[:, start:end] == pd.Series({c:0 for c in lst})).all(axis = 1).sum() == 0 :
        print("X = 1 : 하나라도 1인 게 있음(문제 없음)")
    else :
        print("X = 1 : 전부 0인 게 있음(문제 있음)")

    if (df_train.loc[df_train[col] == 0,].iloc[:, start:end] == pd.Series({c:1 for c in lst})).sum().sum() == 0 :
        print("X = 0 : 전부 0임(문제 없음)")
    else :
        print("X = 0 : 하나라도 1인게 있음(문제 있음)")

In [3]:
lst = []

for i, l in enumerate(df_train) :
    lst.append(df_train[l].unique())

In [4]:
lst2 = []

for i, l in enumerate(df_train) :
    lst2.append(list(df_train[[l]].groupby(l).agg({l : 'count'})[l]))

In [5]:
set(df_concepts.Name) - set(df_train.columns)

{'SkillTrace',
 'SkillTraceErrorIntercept',
 'SkillTraceErrorSlope',
 'SkillTraceScore',
 'SkillTraceTrials'}

**피쳐 설명이 엇갈리는 부분이 있으므로, (https://ludii.games/searchConcepts.php) 를 참고할 것**

In [141]:
df_train.head()

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [144]:
df_train.columns

Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Properties', 'Format',
       'Time', 'Discrete', 'Realtime', 'Turns',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=814)

In [149]:
df_test.iloc[:, [1,2,3]+features_list+list(range(120, 810))]

,GameRulesetName,agent1,agent2,Stochastic,AsymmetricPiecesType,PlayersWithDirections,Team,SquareShape,HexShape,TriangleShape,...,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules
0,00Y,MCTS-UCB1-0.6-NST-false,MCTS-ProgressiveHistory-0.1-MAST-true,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
2,00Y,MCTS-UCB1Tuned-0.1-Random200-true,MCTS-ProgressiveHistory-0.1-MAST-true,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."


In [147]:
features_list = [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
                 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
                 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
                 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121))

In [146]:
df_train.iloc[:,[1,2,3] + [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121)) + list(range(120, 810))]

,GameRulesetName,agent1,agent2,Stochastic,AsymmetricPiecesType,PlayersWithDirections,Team,SquareShape,HexShape,TriangleShape,...,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules
0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."


In [7]:
df_features = pd.DataFrame({'uniqueset' : lst, 'colname' : df_train.columns, 'description' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].Description) + [' ',' ',' ', ' ', ' ', ' '], 'LeafNode' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].LeafNode) + [' ',' ',' ', ' ', ' ', ' '], 'unique_count' : lst2})

Stochastic : 게임에 확률적 요소가 포함됩니다.

Asymmetric : 규칙 및 파워의 비대칭성이 존재합니다.

> AsymmetricForces : 플레이어의 파워가 다릅니다.
>
> > AsymmetricPiecesType : 각 플레이어가 보유한 말의 종류가 다릅니다. (리프 노드)

PlayersWithDirections : 플레이어들이 방향을 사용합니다.

Cooperation : 플레이어들은 협력해야 합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> Team : 게임에는 플레이어 팀이 포함됩니다.

(38) Shape : 게임판의 모양이 존재합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralShape : 나선형 보드입니다.
>
> StarShape : 별 모양 보드입니다.

(54) Tiling : 보드는 타일로 이뤄집니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralTiling : 나선형 타일입니다.

(67) MancalaBoard : 만칼라 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(74) AlquerqueBoard : 앨커키 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(79) ThreeMensMorrisBoard : 쓰리 맨즈 모리스 보드이다.

(87) Track : 보드에 트랙이 존재한다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(93) PlayableSites : 플레이 가능한 구역

> (97) NumPlayableSitesOnBoard : 보드 위에 사용가능한 위치의 수. 여기부터 수치형이 존재한다.

(98) NumColumns : 보드는 n개의 열로 구성되어 있다. ~ (120) NumPhasesBoard : 보드 변화 페이즈

[98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120]

In [136]:
[12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121)) + [
     
 ]

[12,
 21,
 28,
 30,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 47,
 49,
 51,
 52,
 55,
 56,
 57,
 59,
 61,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120]

In [86]:
df_train.utility_agent1.mean()

0.04439766262399326

In [89]:
df_train.loc[df_train.AlquerqueBoardWithEightTriangles == 1,].utility_agent1.mean()

0.013333333333333324

In [94]:
df_train.loc[df_train.ThreeMensMorrisBoard == 0,].ThreeMensMorrisBoardWithTwoTriangles.sum()

0

In [124]:
checking_set('Track', 88, 90)

X = 1 : 전부 0인 게 있음(문제 있음)
X = 0 : 전부 0임(문제 없음)


In [137]:
df_features.iloc[121:140, ]

,uniqueset,colname,description,LeafNode,unique_count
121,"[0, 1]",Hand,Number of component types.,1,"[141166, 92068]"
122,"[1, 4, 3, 2]",NumContainers,Average number of component types per player.,1,"[133540, 17352, 73896, 8446]"
123,"[48, 8, 24, 33, 102, 62, 65, 60, 63, 18, 121, ...",NumPlayableSites,Number of dice.,1,"[184, 102, 206, 684, 1644, 870, 4426, 4036, 33..."
124,[1],Component,Rules of the game.,0,[233234]
125,"[1, 0]",Piece,Global metarules that override all other rules.,0,"[1876, 231358]"
126,"[0, 1]",PieceValue,Game involves an opening round equaliser.,0,"[227672, 5562]"
127,"[0, 1]",PieceRotation,Second player may swap colours.,1,"[232900, 334]"
128,"[0, 1]",PieceDirection,Game has repetition checks.,0,"[202030, 31204]"
129,"[0, 1]",Dice,No repeated piece positions within a single turn.,1,"[217358, 15876]"
130,"[0, 1]",DiceD2,No repeated states withing a single turn.,1,"[226452, 6782]"
